In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import arviz as az

WARNING (theano.link.c.cmodule): install mkl with `conda install mkl-service`: No module named 'mkl'
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [15]:
model = pm.Model()

with model:
    cutremur = pm.Bernoulli('C', 0.0005)
    incendiu = pm.Deterministic('I', pm.math.switch(cutremur, 0.03, 0.01))
#     este cutremur si incendiu: 98%
#     este cutremur si nu este incendiu:  2%
#     fara cutremur si este incendiu: 95%
#     fara cutremur si fara incendiu: 0.01%
# cumparat produs <=> alarma de incendiu declansata
# sa fi avut nevoie urgenta de produs  <=> sa fi avut loc un cutremur
    alarm_p = pm.Deterministic('FA_p', pm.math.switch(cutremur, pm.math.switch(incendiu, 0.98, 0.02), pm.math.switch(incendiu, 0.95, 0.0001)))
    alarm = pm.Bernoulli('FA', p= alarm_p, observed = 1)
    trace = pm.sample(20000)
    
dictionary = {
              'cutremur': trace['C'].tolist(),
              'incendiu': trace['I'].tolist(),
              }
df = pd.DataFrame(dictionary)
print("good", df[(df['cutremur'] == 1)].shape[0])
print("possible:", df.shape[0])
p_cutremur = df[(df['cutremur'] == 1)].shape[0] / df.shape[0]
print("P(Cutremur | Alarma):", p_cutremur)

/srv/conda/envs/notebook/lib/python3.7/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Multiprocess sampling (4 chains in 4 jobs)
BinaryGibbsMetropolis: [C]


Sampling 4 chains for 1_000 tune and 20_000 draw iterations (4_000 + 80_000 draws total) took 40 seconds.


good 41
possible: 80000
P(Cutremur | Alarma): 0.0005125


In [17]:
model2 = pm.Model()

with model2:
    cutremur = pm.Bernoulli('C', 0.0005)
    incendiu_p = pm.Deterministic('I_p', pm.math.switch(cutremur, 0.03, 0.01))
    incendiu = pm.Bernoulli('I', p = incendiu_p)
    
    alarm_p = pm.Deterministic('FA_p', pm.math.switch(cutremur, pm.math.switch(incendiu, 0.98, 0.02), pm.math.switch(incendiu, 0.95, 0.0001)))
    alarm = pm.Bernoulli('A', p= alarm_p)
    trace2 = pm.sample(20000)
    
dictionary2 = {
                'cutremur': trace2['C'].tolist(),
                'incendiu': trace2['I'].tolist(),
                'alarm': trace2['A'].tolist()
                }
        
df2 = pd.DataFrame(dictionary2)

p_incendiu_fara_alarma = df2[((df2['incendiu'] == 1) & (df2['alarm'] ==0))].shape[0] / df2.shape[0]

print("P(Incendiu & !Alarma):", p_incendiu_fara_alarma)


/srv/conda/envs/notebook/lib/python3.7/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Multiprocess sampling (4 chains in 4 jobs)
BinaryGibbsMetropolis: [C, I, A]


Sampling 4 chains for 1_000 tune and 20_000 draw iterations (4_000 + 80_000 draws total) took 53 seconds.
The number of effective samples is smaller than 10% for some parameters.


P(Incendiu & !Alarma): 0.0005875
